
# CortexRAG - End to End RAG with 3 lines of code

#### Standard Snowflake Session Initialization

In [1]:
from snowvecdb import SnowVectorDB
from snowrag import SnowRAG
from snowflake.snowpark import Session
import os
from dotenv import load_dotenv
load_dotenv()

connection_parameters = {
    
    "account": os.getenv('SNOWFLAKE_ACCOUNT'),
    "user": os.getenv('SNOWFLAKE_USER'),
    "password": os.getenv('SNOWFLAKE_PASSWORD'),
    "role": os.getenv('SNOWFLAKE_ROLE'),
    "warehouse": os.getenv('SNOWFLAKE_WAREHOUSE'),
    "database": os.getenv('SNOWFLAKE_DATABASE'),
    "schema": os.getenv('SNOWFLAKE_SCHEMA')}  

session = Session.builder.configs(connection_parameters).create()

### Create Embeddings Table from local directory

In [3]:
SVDB = SnowVectorDB(snowflake_session=session) 
SVDB("ANNUAL_REPORT_EMBEDDINGS",data_source_directory="YOUR_LOCAL_DATA SOURCE_DIRECTORY/sample_documents")

Loading source data...
...data load complete.
Creating embeddings table...
Embeddings succesfully loaded in Snowflake table:ANNUAL_REPORT_EMBEDDINGS


### Create Embeddings Table from files in an existing stage

In [8]:
SVDB("ANNUAL_REPORT_EMBEDDINGS_STG",stage="FD_EMBEDDINGS_RAG")

Creating embeddings table...
Embeddings succesfully loaded in Snowflake table:ANNUAL_REPORT_EMBEDDINGS_STG


### Initialize RAG Chat Bot

In [4]:
rag = SnowRAG(embeddings_table="ANNUAL_REPORT_EMBEDDINGS",lm_model="mixtral-8x7b",snowflake_session=session)

### Run the program

In [7]:
rag("How many customers did Snowflake have in fiscal year 2024?")

[' 1. Snowflake had 9,437 total customers as of January 31, 2024.\n2. The fiscal year 2024 for Snowflake ended on January 31, 2024.\n3. The total customer count includes adjustments for acquisitions, consolidations, spin-offs, and other market activity.\n\nSo, based on the provided information, Snowflake had 9,437 customers in fiscal year 2024.']